### Importing of Modules

In [3]:
import pandas as pd
import json
import requests
import numpy as np

C:\Users\junke\AppData\Local\Temp\ipykernel_14232\913430442.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Fetching Data 

In [4]:
"""
Fetch JSON data from the given URL
"""
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad responses
parsed_data = response.json() 

In [5]:
"""
Unedited Parsed Restaurant Data
"""
pd.json_normalize(parsed_data).head()

# The 'restaurants' column contains nested data, which includes restaurant details. I will expand upon the nested data.

,results_found,results_start,results_shown,restaurants
0,29287,1,20,"[{'restaurant': {'R': {'res_id': 18649486}, 'a..."
1,7625,1,20,"[{'restaurant': {'R': {'res_id': 18707652}, 'a..."
2,21776,1,20,"[{'restaurant': {'R': {'res_id': 18392725}, 'a..."
3,16762,1,20,"[{'restaurant': {'R': {'res_id': 58882}, 'apik..."
4,12026,1,20,"[{'restaurant': {'R': {'res_id': 18893197}, 'a..."


In [6]:
"""
Normalize restaurant Data
"""
main_restaurant_df = pd.json_normalize(parsed_data, "restaurants")

In [7]:
"""
Read in Country-Code Excel file into DataFrame
"""

country_code_df = pd.read_csv("Country-Code.csv")

### Part 1

In [8]:
"""
Preview of country_code_df
"""

country_code_df.head()

,Country Code,Country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia


In [9]:
"""
Preview of main_restaurant_df
"""

main_restaurant_df.head(2)

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,...,restaurant.has_table_booking,restaurant.events_url,restaurant.establishment_types,restaurant.medio_provider,restaurant.order_url,restaurant.order_deeplink,restaurant.book_url,restaurant.user_rating.custom_rating_text,restaurant.user_rating.custom_rating_text_background,restaurant.user_rating.rating_tool_tip
0,18649486,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1,28.4936741035,...,0,https://www.zomato.com/ncr/the-drunken-botanis...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,308322,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1,28.5542935327,...,1,https://www.zomato.com/HauzKhasSocial/events#t...,[],1,https://www.zomato.com/HauzKhasSocial/order?ut...,,https://www.zomato.com/HauzKhasSocial/book?utm...,NaN,NaN,NaN


In [10]:
"""
Merge Main restaurant data with country code DataFrame based on country ID.
"""

restaurant_countrycode_df = pd.merge(
    main_restaurant_df, country_code_df, 
    how='left', left_on='restaurant.location.country_id', 
    right_on='Country Code'
    )

restaurant_countrycode_df.head(3)

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,...,restaurant.establishment_types,restaurant.medio_provider,restaurant.order_url,restaurant.order_deeplink,restaurant.book_url,restaurant.user_rating.custom_rating_text,restaurant.user_rating.custom_rating_text_background,restaurant.user_rating.rating_tool_tip,Country Code,Country
0,18649486,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1,28.4936741035,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,India
1,308322,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1,28.5542935327,...,[],1,https://www.zomato.com/HauzKhasSocial/order?ut...,,https://www.zomato.com/HauzKhasSocial/book?utm...,NaN,NaN,NaN,1.0,India
2,18856789,cba15beb4c265876a9828f242b4cf41c,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,"1/83, Third Floor, Club Road, West Punjabi Bag...",Punjabi Bagh,New Delhi,1,28.5478090000,...,[],,NaN,NaN,https://www.zomato.com/ncr/air-an-ivory-region...,NaN,NaN,NaN,1.0,India


In [11]:
"""
Extract required fields from the merged DataFrame and rename these columns
"""

# Extraction of columns
extract_restaurant_df = restaurant_countrycode_df[[
    'restaurant.R.res_id', 'restaurant.name', 'Country', 
    'restaurant.location.city', 'restaurant.user_rating.votes', 
    'restaurant.user_rating.aggregate_rating', 'restaurant.cuisines'
    ]]

# Dictionary of columns to rename
renamed_columns = {
    'restaurant.R.res_id': 'Restaurant Id',
    'restaurant.name': 'Restaurant Name',
    'restaurant.location.city': 'City',
    'restaurant.user_rating.votes': 'User Rating Votes',
    'restaurant.user_rating.aggregate_rating': 'User Aggregate Rating',
    'restaurant.cuisines': 'Cuisines'
}

# Rename required columns
renamed_restaurant_df = extract_restaurant_df.rename(columns=renamed_columns)

In [12]:

"""
Convert data type of the "User Aggregate Rating" column in the DataFrame.
"""

renamed_restaurant_df['User Aggregate Rating'] = renamed_restaurant_df['User Aggregate Rating'].astype('float64')

In [40]:
"""
To check for duplicates
"""

renamed_restaurant_df.duplicated().sum()

0

In [42]:
"""
To check for null values
"""
renamed_restaurant_df.isna().sum()

Restaurant Id             0
Restaurant Name           0
Country                  20
City                      0
User Rating Votes         0
User Aggregate Rating     0
Cuisines                  0
dtype: int64

In [41]:
"""
Investigate Null values
"""

renamed_restaurant_df[final_restaurant_df.Country.isna()].head()

,Restaurant Id,Restaurant Name,Country,City,User Rating Votes,User Aggregate Rating,Cuisines
1140,7210754,The Golconda Bowl,NaN,Dummy,0,0.0,"North Indian, Mughlai"
1141,7204069,California Cantina,NaN,Dummy,0,0.0,"Burger, Fast Food, Mexican, American, Pizza"
1142,7200516,Bardelli's,NaN,Dummy,0,0.0,South Indian
1143,18133267,Baton Rouge,NaN,Dummy,0,0.0,Steak
1144,7204711,Dunkin Donuts,NaN,Dummy,0,0.0,"Fast Food, Desserts"


In [43]:
"""
Dropping of Null values
"""

# Since the rows with null values seem to be dummy restaurants, I have opted to drop them
final_restaurant_df = renamed_restaurant_df.dropna()

In [13]:
"""
Preview of final dataframe
"""

final_restaurant_df.head()

,Restaurant Id,Restaurant Name,Country,City,User Rating Votes,User Aggregate Rating,Cuisines
0,18649486,The Drunken Botanist,India,Gurgaon,4765,4.4,"Continental, Italian, North Indian, Chinese"
1,308322,Hauz Khas Social,India,New Delhi,13627,4.6,"Continental, American, Asian, North Indian, Ch..."
2,18856789,AIR- An Ivory Region,India,New Delhi,1819,4.1,"North Indian, Chinese, Continental, Asian"
3,307374,AMA Cafe,India,New Delhi,3252,4.4,"Cafe, Juices"
4,18238278,Tamasha,India,New Delhi,8112,4.4,"Finger Food, North Indian, Continental, Italian"


In [14]:
"""
Export final_restaurant_df to CSV
"""

final_restaurant_df.to_csv(input(), index=False)

# My input was C:\\Users\\junke\\Desktop\\Important Documents (JunKeat)\\GovTech-Application\\restaurants.csv

### Part 2

In [15]:
"""
Preview of the data in zomato_events
"""

main_restaurant_df["restaurant.zomato_events"][1117]

# The 'restaurants' column contains a list of dictionaries, with each dictionary representing
# events associated with a single restaurant. This nested structure stores multiple events
# pertaining to individual restaurants within the same entry of the DataFrame.

[{'event': {'event_id': 305850,
   'friendly_start_date': '01 March',
   'friendly_end_date': '30 April',
   'friendly_timing_str': 'Friday, 1st March - Tuesday, 30th April',
   'start_date': '2019-03-01',
   'end_date': '2019-04-30',
   'end_time': '22:00:00',
   'start_time': '17:30:00',
   'is_active': 1,
   'date_added': '2019-01-31 07:00:12',
   'photos': [{'photo': {'url': 'https://b.zmtcdn.com/data/zomato_events/photos/c32/6f3e2292cfd37f13698e650113679c32_1548898348.jpg',
      'thumb_url': 'https://b.zmtcdn.com/data/zomato_events/photos/c32/6f3e2292cfd37f13698e650113679c32_1548898348.jpg?fit=around%7C100%3A100&crop=100%3A100%3B%2A%2C%2A',
      'order': 0,
      'md5sum': '6f3e2292cfd37f13698e650113679c32',
      'id': 412548,
      'photo_id': 412548,
      'uuid': 1726901202,
      'type': 'NORMAL'}}],
   'restaurants': [],
   'is_valid': 1,
   'share_url': 'http://www.zoma.to/r/0',
   'show_share_url': 0,
   'title': 'Smoky Thursdays',
   'description': "We're adding some mo

In [16]:
"""
Expanding the restaurant.zomato_events column 
"""

# Unpack the lists in the events column vertically
expended_events_df = main_restaurant_df.explode("restaurant.zomato_events")

# Unpack the dictionaries within each element of the column
unpacked_events_df = pd.json_normalize(expended_events_df["restaurant.zomato_events"])

# Merge the unpacked DataFrame with the original DataFrame
events_df = pd.concat([main_restaurant_df, unpacked_events_df], axis=1)

# Result 
events_df.head(2)

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,...,event.display_date,event.is_end_time_set,event.disclaimer,event.event_category,event.event_category_name,event.book_link,event.types,event.share_data.should_show,event.is_zomato_event,event.url
0,18649486.0,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1.0,28.4936741035,...,06 March - 28 August,0.0,Restaurants are solely responsible for the ser...,1.0,,,[],0.0,NaN,NaN
1,308322.0,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1.0,28.5542935327,...,29 March - 23 May,1.0,Restaurants are solely responsible for the ser...,0.0,,,[],0.0,NaN,NaN


In [17]:
"""
Filtering past event in the month of April 2019 
"""

# Change event start and end date to a date data type
events_df['event.start_date'] = pd.to_datetime(events_df["event.start_date"])
events_df['event.end_date'] = pd.to_datetime(events_df["event.end_date"])


# Define the condition for events starting in or before April 2019
start_date_condition = (
    (events_df['event.start_date'].dt.year == 2019) & 
    (events_df['event.start_date'].dt.month <= 4)
)

# Define the condition for events ending in or after April 2019
end_date_condition = (
    (events_df['event.end_date'].dt.year == 2019) & 
    (events_df['event.end_date'].dt.month >= 4)
)

# Apply the conditions to filter the DataFrame
April2019_events_df = events_df[(start_date_condition | end_date_condition)]

# To check filtered results
April2019_events_df.head()

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,...,event.display_date,event.is_end_time_set,event.disclaimer,event.event_category,event.event_category_name,event.book_link,event.types,event.share_data.should_show,event.is_zomato_event,event.url
0,18649486.0,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1.0,28.4936741035,...,06 March - 28 August,0.0,Restaurants are solely responsible for the ser...,1.0,,,[],0.0,NaN,NaN
1,308322.0,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1.0,28.5542935327,...,29 March - 23 May,1.0,Restaurants are solely responsible for the ser...,0.0,,,[],0.0,NaN,NaN
2,18856789.0,cba15beb4c265876a9828f242b4cf41c,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,"1/83, Third Floor, Club Road, West Punjabi Bag...",Punjabi Bagh,New Delhi,1.0,28.5478090000,...,10 April - 11 April,1.0,Restaurants are solely responsible for the ser...,0.0,,,"[{'name': 'Bollywood Night', 'color': '#6454B8'}]",0.0,NaN,NaN
7,18382360.0,cba15beb4c265876a9828f242b4cf41c,18382360,Local,https://www.zomato.com/ncr/local-connaught-pla...,"11, KG Marg, Scindia House, Connaught Place, N...",Connaught Place,New Delhi,1.0,28.6299140703,...,10 April,1.0,Restaurants are solely responsible for the ser...,0.0,,,"[{'name': 'Cocktail Mixology', 'color': '#C32A...",0.0,NaN,NaN
8,18273624.0,cba15beb4c265876a9828f242b4cf41c,18273624,Cafeteria & Co.,https://www.zomato.com/ncr/cafeteria-co-vijay-...,"G 14, Hudson Lane, Vijay Nagar, New Delhi",Vijay Nagar,New Delhi,1.0,28.6944242830,...,10 April,1.0,Restaurants are solely responsible for the ser...,0.0,,,"[{'name': 'Cocktail Mixology', 'color': '#C32A...",0.0,NaN,NaN


In [18]:
"""
Extraction and renaming of required columns
"""

# Extract required Columns
April2019_events_df = April2019_events_df[[
    "event.event_id", "restaurant.id",
    "restaurant.name", "restaurant.photos_url",
    "event.title", "event.start_date",
    "event.end_date"
    ]]

# Rename required columns
columns_to_rename = {
    "event.event_id":"Event Id", 
    "restaurant.id":"Restaurant Id",
    "restaurant.name":"Restaurant Name", 
    "restaurant.photos_url":"Photo URL",
    "event.title":"Event Title", 
    "event.start_date":"Event Start Date",
    "event.end_date":"Event End Date"
}

final_events_df = April2019_events_df.rename(columns=columns_to_rename)

In [45]:
"""
To check for null values
"""

final_events_df.isna().sum()

Event Id            0
Restaurant Id       0
Restaurant Name     0
Photo URL           0
Event Title         0
Event Start Date    0
Event End Date      0
dtype: int64

In [47]:
"""
To check duplicate values
"""

final_events_df.duplicated().sum()

0

In [19]:
"""
Export final_events_df to CSV
"""

final_events_df.to_csv(input(), index=False)

# My input was C:\\Users\\junke\\Desktop\\Important Documents (JunKeat)\\GovTech-Application\\restaurant_events.csv

### Part 3

In [20]:
"""
Preview of main_restaurant_df
"""

main_restaurant_df.head()

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,...,restaurant.has_table_booking,restaurant.events_url,restaurant.establishment_types,restaurant.medio_provider,restaurant.order_url,restaurant.order_deeplink,restaurant.book_url,restaurant.user_rating.custom_rating_text,restaurant.user_rating.custom_rating_text_background,restaurant.user_rating.rating_tool_tip
0,18649486,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1,28.4936741035,...,0,https://www.zomato.com/ncr/the-drunken-botanis...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,308322,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1,28.5542935327,...,1,https://www.zomato.com/HauzKhasSocial/events#t...,[],1,https://www.zomato.com/HauzKhasSocial/order?ut...,,https://www.zomato.com/HauzKhasSocial/book?utm...,NaN,NaN,NaN
2,18856789,cba15beb4c265876a9828f242b4cf41c,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,"1/83, Third Floor, Club Road, West Punjabi Bag...",Punjabi Bagh,New Delhi,1,28.5478090000,...,1,https://www.zomato.com/ncr/air-an-ivory-region...,[],,NaN,NaN,https://www.zomato.com/ncr/air-an-ivory-region...,NaN,NaN,NaN
3,307374,cba15beb4c265876a9828f242b4cf41c,307374,AMA Cafe,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,"House 6, New Colony, Majnu ka Tila, New Delhi",Majnu ka Tila,New Delhi,1,28.7025817618,...,0,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238278,cba15beb4c265876a9828f242b4cf41c,18238278,Tamasha,https://www.zomato.com/ncr/tamasha-connaught-p...,"28, Block A, Kasturba Gandhi Marg, Connaught P...",Connaught Place,New Delhi,1,28.6296624581,...,0,https://www.zomato.com/ncr/tamasha-connaught-p...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
"""
Filter for the specified rating texts only
"""

specified_texts = ['Excellent', 'Very Good', 'Good', 'Average', 'Poor']
filtered_rating_df = main_restaurant_df[main_restaurant_df['restaurant.user_rating.rating_text'].isin(specified_texts)]

In [61]:
"""
To check for null values in the aggregate rating column"""

filtered_rating_df['restaurant.user_rating.aggregate_rating'].isna().sum()

0

In [62]:
"""
Changing the aggregate rating column to float data type
"""

filtered_rating_df['restaurant.user_rating.aggregate_rating'] = filtered_rating_df['restaurant.user_rating.aggregate_rating'].astype("float64")

C:\Users\junke\AppData\Local\Temp\ipykernel_14232\1870301180.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rating_df['restaurant.user_rating.aggregate_rating'] = filtered_rating_df['restaurant.user_rating.aggregate_rating'].astype("float64")


In [67]:
"""
Analyze the distribution of aggregate ratings for each rating text
"""

aggregate = [ 'min', 'max']

rating_statistics = filtered_rating_df.groupby('restaurant.user_rating.rating_text')['restaurant.user_rating.aggregate_rating'].agg(aggregate)

rating_statistics

,min,max
restaurant.user_rating.rating_text,,
Average,2.5,3.4
Excellent,4.5,4.9
Good,3.5,3.9
Poor,2.2,2.2
Very Good,4.0,4.4


In [69]:
"""
Export rating_statistics to json
"""

rating_statistics.to_json(input(), index=False)

# My input was C:\\Users\\junke\\Desktop\\Important Documents (JunKeat)\\GovTech-Application\\restaurant_data.json

In [37]:
pd.set_option('display.max_columns', 500)